In [1]:
#1
%matplotlib inline
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from consts import sensor_day_seconds as day_second
from consts import sensor_idd as idd
from consts import sensor_idds_uniq as idds_uniq
from consts import sensor_idds_uniq_sorted as idds_uniq_sorted
from consts import sensor_a_temperature as a_temperature
from consts import sensor_s_temperature as s_temperature
from consts import sensor_unix_column as unix_column
from consts import sensor_idd as idd
epoch_length = 30
import multiprocessing as mp
from multiprocessing import Pool
import time

In [2]:
#2
input_file = open('dict_list3_1.pkl','rb')
input_list = pickle.load(input_file)
input_file.close()

In [3]:
#3
# 将epoch epoch_len epoch_time 都flat化
len_list = []
time_stamp_list = []
epoch_list = []
for start_time in input_list:
    # print(len(start_time[0]))
    for dict in start_time[0]:
        len_list.append(len(dict))
        epoch_list.append(dict)
    start_point = start_time[1]
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    time_stamp_list.extend(time_stamps[:-1])
len_array = np.array(len_list)
time_stamp_array = np.array(time_stamp_list) # 时间序列
epoch_array = np.array(epoch_list)

/home/esmidth/.pyenv/versions/miniconda3-4.7.12/envs/ccs_ta/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


In [ ]:
start_list = np.arange(1162393768,1178747998+day_second,day_second)
ddf_list = []
for start_point in start_list:
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    sub_df_list = []
    df = pd.read_hdf('day2.h5',str(start_point))
    for i in range(len(time_stamps)-1):
        sub_df = df[(df[unix_column] > time_stamps[i]) & (df[unix_column] <= time_stamps[i+1])]
        sub_df_list.append(sub_df)
    ddf_list.extend(sub_df_list)

In [ ]:
# 修改为多线程化读入

In [4]:
#4
def read_sub_df(start_point):
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    sub_df_list = []
    df = pd.read_hdf('day2.h5',str(start_point))
    for i in range(len(time_stamps)-1):
        sub_df = df[(df[unix_column] > time_stamps[i]) & (df[unix_column] <= time_stamps[i+1])]
        sub_df_list.append(sub_df)
    print("Done \t {}".format(start_point))
    return (start_point,sub_df_list)

In [5]:
#5
start_list = np.arange(1162393768,1178747998+day_second,day_second)
manager = mp.Manager()
# aggreateData = manager.dict()
p = Pool(16)
tmp_result = []

for start_point in start_list:
    result = p.apply_async(func=read_sub_df,args=(start_point,))
    tmp_result.append(result)
p.close()
p.join()


Done 	 1162825768
Done 	 1162480168
Done 	 1162393768Done 	 1162998568
Done 	 1162652968
Done 	 1162739368Done 	 1162566568


Done 	 1162912168
Done 	 1163171368
Done 	 1163516968
Done 	 1163257768
Done 	 1163344168
Done 	 1163430568
Done 	 1163603368
Done 	 1163689768
Done 	 1163084968
Done 	 1163776168
Done 	 1163862568
Done 	 1163948968
Done 	 1164035368
Done 	 1164121768
Done 	 1164208168
Done 	 1164294568
Done 	 1164380968
Done 	 1164467368
Done 	 1164553768
Done 	 1164640168
Done 	 1164726568
Done 	 1164812968
Done 	 1164899368
Done 	 1164985768
Done 	 1165072168
Done 	 1165158568
Done 	 1165244968
Done 	 1165331368
Done 	 1165417768
Done 	 1165504168
Done 	 1165590568
Done 	 1165676968
Done 	 1165763368
Done 	 1165849768
Done 	 1165936168
Done 	 1166022568
Done 	 1166108968
Done 	 1166195368
Done 	 1166281768
Done 	 1166368168
Done 	 1166454568
Done 	 1166540968
Done 	 1166627368
Done 	 1166713768
Done 	 1166800168
Done 	 1166886568
Done 	 1166972968
Done 	 1167059368
Done 	 116

In [6]:
#6
origin_time = start_list[0]

results = []
for i in start_list:
    results.append([])

for item in tmp_result:
    temp = item.get()
    results[int((temp[0]-origin_time)/86400)].extend(temp[1])

ddf_list = []
for item in results:
    ddf_list.extend(item)

In [7]:
#7
index1 = np.where(len_array > 40)[0]
maybe_array = epoch_array[index1]

In [8]:
#8
# init maybe_dict 
maybe_dict = {}
for i in idds_uniq:
    maybe_dict[i] = 0


for item in maybe_array:
    uniq_item = np.unique(item)
    for j in uniq_item:
        maybe_dict[j] += 1
print(maybe_dict)

{7: 116368, 8: 311230, 11: 311649, 14: 297530, 17: 311626, 19: 295072, 23: 309388, 26: 309247, 27: 77054, 31: 299208, 33: 267283, 34: 316424, 35: 274661, 40: 266279, 46: 314496, 47: 114937, 49: 308374, 51: 318062, 55: 138159, 57: 224804, 59: 216308, 60: 292967, 61: 231792, 62: 263384, 63: 208200, 65: 294875, 69: 198539, 70: 316076, 71: 203416, 72: 301465, 73: 306707, 75: 303260, 76: 310711, 79: 273899, 80: 295663, 81: 251044, 82: 131717, 84: 308895, 87: 315132, 88: 234990, 89: 283497, 93: 302051, 96: 313535, 97: 310211, 98: 221392, 100: 306061, 103: 318015, 104: 313801, 109: 190240, 111: 303233, 121: 302922}


In [9]:
#9
maybe_dict_sorted = sorted(maybe_dict.items(),key=lambda kv:(kv[1],kv[0]),reverse=True)
maybe_dict_sorted = np.array(maybe_dict_sorted)

In [ ]:
sub_sensor_list = maybe_dict_sorted.T[0]
print(sub_sensor_list)

In [10]:
#10
# 测试sensor数量为多少合适
def get_sub_index(sensor_count):
    sub_sensor_list = idds_uniq_sorted[:sensor_count]
    sub_index = []
    for i, item in enumerate(epoch_array):
        uniq_item = np.unique(item)
        diff = np.setdiff1d(sub_sensor_list,uniq_item)
        if len(diff) == 0:
            sub_index.append(i)
    return len(sub_index),sub_index

In [11]:
#11 多进程版 
start_time = time.time()
def multi_get_sub_index(i):
    return get_sub_index(i)

sensor_count_list = np.linspace(10,50,41,dtype=int)

manager = mp.Manager()
# aggreateData = manager.dict()
p = Pool(16)
tmp_result = []

for i in sensor_count_list:
    result = p.apply_async(func=multi_get_sub_index,args=(i,))
    tmp_result.append(result)
p.close()
p.join()
print(time.time()-start_time)

79.61964082717896


In [13]:
#11-1
sub_index_list = []
for item in tmp_result:
    sub_index_list.append(item.get())
sub_index_array = np.array(sub_index_list)

/home/esmidth/.pyenv/versions/miniconda3-4.7.12/envs/ccs_ta/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [12]:
#12
def get_length_count(window):
    for item in sub_index_array:
        zeros = np.zeros_like(epoch_array)
        zeros[item[1]] = 1
        sum = 0
        for j in range(len(zeros)):
            tmp = zeros[j:j+window]
            if 0 not in tmp:
                sum += 1
        print(sum)

In [15]:
# 12-1 DUMP
output_file = open('sub_index_array_drop_na.pkl','wb')
pickle.dump(sub_index_array,output_file)
output_file.close()

In [ ]:
# 12-2 LOAD
input_file = open('sub_index_array_drop_na.pkl','rb')
sub_index_array = pickle.load(input_file)
input_file.close()

In [14]:
#13
def get_data_matrixs(sensor_count,window,sub_index_array = None):
    print(idds_uniq_sorted[:sensor_count])
    tmp = sub_index_array[sensor_count-10][1]
    zeros = np.zeros_like(epoch_array)
    zeros[tmp] = 1

    sum_list = []
    for i in range(len(zeros)):
        tmp1 = zeros[i:i+window]
        if 0 not in tmp1:
            sum_list.append(i)
            
    return sum_list
indices = get_data_matrixs(19,7,sub_index_array)

[51, 103, 34, 70, 87, 46, 104, 96, 11, 17, 8, 76, 97, 23, 26, 84, 49, 73, 100]


In [15]:
#14
# 丢弃多余帧
def drop_useless_sensor(df,sensor_list):
    # print(sensor_list)
    for i in df[idd].values:
        if i not in sensor_list:
            df = df[df[idd] != i]
    # print(len(df))
    if len(df) < len(sensor_list):
        return None
    return df
# t = drop_useless_sensor(ddf_list[indices[3]],idds_uniq_sorted[:19])
# print(len(t))

In [ ]:
# ignore
def handle_df(ddf_list,indice,sensor_list):
    df = ddf_list[indice]
    df = df.drop_duplicates(idd)
    df = drop_useless_sensor(df,sensor_list)
    return df

df_sample = handle_df(ddf_list,indices[0],idds_uniq_sorted[:19])

In [24]:
# before 15
for i, df in enumerate(ddf_list):
    with open('./data/{}.pkl'.format(i),'wb') as f:
        pickle.dump(df,f)

In [25]:
#15 multi_processing
def multi_convert_df_to_dm(indice,sensor_list,window=7):
    df_list = []
    for i in range(window):
        with open('./data/{}.pkl'.format(indice+i),'rb') as f:
            sub_df = pickle.load(f)
        df_list.append(sub_df)
    return_list = []
    for df in df_list:
        df = df.drop_duplicates(idd)
        df = drop_useless_sensor(df,sensor_list)
        if type(df) == type(None):
            break
        df = df.sort_values(by=idd)
        flat_matrix = np.zeros(100)
        indice_dict = {63:10,23:14,17:15,47:20,7:21,27:22,26:24,14:25,121:26,46:27,8:28,111:32,75:33,57:34,59:35,40:36,34:37,61:41,71:42,65:43,82:45,62:46,76:48,51:51,11:52,70:53,49:54,60:57,84:60,81:61,55:62,72:63,73:64,35:66,31:67,69:68,79:69,103:70,80:71,96:72,97:75,100:76,33:77,19:78,93:79,109:80,87:81,104:84,89:91,88:92,98:94}
        return_list.append(df[s_temperature].values)
    return np.array(return_list).T
res_array = multi_convert_df_to_dm(indices[46],idds_uniq_sorted[:19],7)
print(res_array)

[[12.412 12.537 12.412 12.537 12.475 12.787 12.787]
 [13.287 13.35  13.225 13.225 13.1   13.412 13.287]
 [13.912 14.037 14.037 13.912 13.85  14.1   14.162]
 [12.662 12.725 12.725 12.725 12.6   12.6   12.725]
 [13.225 13.537 13.412 13.35  13.35  13.287 13.225]
 [12.912 12.975 12.85  12.85  12.975 12.912 12.85 ]
 [11.787 11.662 11.725 11.725 11.787 11.662 11.412]
 [13.1   12.85  12.975 12.975 12.912 12.85  13.037]
 [13.475 13.412 13.537 13.537 13.537 13.412 13.662]
 [12.85  12.975 12.725 12.787 12.85  12.787 12.975]
 [13.537 13.662 13.85  13.912 13.975 13.787 13.85 ]
 [13.662 13.787 13.725 13.662 13.725 13.725 13.6  ]
 [12.662 12.787 12.787 12.85  12.725 12.85  12.662]
 [13.787 13.85  13.912 13.912 13.912 13.975 13.912]
 [12.787 12.662 12.85  12.85  12.912 12.975 13.1  ]
 [13.287 13.287 13.475 13.35  13.35  13.6   13.6  ]
 [13.287 13.287 13.35  13.162 13.412 13.537 13.475]
 [13.662 13.662 13.725 13.662 13.662 13.787 13.85 ]
 [12.475 12.475 12.412 12.475 12.475 12.475 12.475]]


In [17]:
#15
def convert_df_to_dm(ddf_list,indice,sensor_list,window=7):
    # print(sensor_list)
#     print(len(sensor_list))
    df_list = []
    for i in range(window):
        df_list.append(ddf_list[indice+i])
#     df = ddf_list[indice]
    return_list = []
    for df in df_list:
        df = df.drop_duplicates(idd)
#         print(indice,type(df))
#         print(len(df))
        df = drop_useless_sensor(df,sensor_list)
        if type(df) == type(None):
            break
        df = df.sort_values(by=idd)
        flat_matrix = np.zeros(100)
        indice_dict = {63:10,23:14,17:15,47:20,7:21,27:22,26:24,14:25,121:26,46:27,8:28,111:32,75:33,57:34,59:35,40:36,34:37,61:41,71:42,65:43,82:45,62:46,76:48,51:51,11:52,70:53,49:54,60:57,84:60,81:61,55:62,72:63,73:64,35:66,31:67,69:68,79:69,103:70,80:71,96:72,97:75,100:76,33:77,19:78,93:79,109:80,87:81,104:84,89:91,88:92,98:94}
        return_list.append(df[s_temperature].values)
#         for i,indice in enumerate(df[idd].values):
#             flat_matrix[indice_dict[indice]] = df[s_temperature].values[i]
    return np.array(return_list).T
    

res_array = convert_df_to_dm(ddf_list,indices[45],idds_uniq_sorted[:19],7)
print(res_array)

[[12.6   12.412 12.537 12.412 12.537 12.475 12.787]
 [13.287 13.287 13.35  13.225 13.225 13.1   13.412]
 [13.787 13.912 14.037 14.037 13.912 13.85  14.1  ]
 [12.6   12.662 12.725 12.725 12.725 12.6   12.6  ]
 [13.225 13.225 13.537 13.412 13.35  13.35  13.287]
 [12.975 12.912 12.975 12.85  12.85  12.975 12.912]
 [11.787 11.787 11.662 11.725 11.725 11.787 11.662]
 [12.912 13.1   12.85  12.975 12.975 12.912 12.85 ]
 [13.287 13.475 13.412 13.537 13.537 13.537 13.412]
 [12.912 12.85  12.975 12.725 12.787 12.85  12.787]
 [13.475 13.537 13.662 13.85  13.912 13.975 13.787]
 [13.85  13.662 13.787 13.725 13.662 13.725 13.725]
 [12.537 12.662 12.787 12.787 12.85  12.725 12.85 ]
 [13.725 13.787 13.85  13.912 13.912 13.912 13.975]
 [12.787 12.787 12.662 12.85  12.85  12.912 12.975]
 [13.412 13.287 13.287 13.475 13.35  13.35  13.6  ]
 [13.287 13.287 13.287 13.35  13.162 13.412 13.537]
 [13.85  13.662 13.662 13.725 13.662 13.662 13.787]
 [12.35  12.475 12.475 12.412 12.475 12.475 12.475]]


In [ ]:
for i,df in enumerate(ddf_list):
    print(i)

In [ ]:
res_array

In [27]:
#16
start_time = time.time()
array_list2 = []
for i in indices:
    array_list2.append(convert_df_to_dm(ddf_list,i,idds_uniq_sorted[:19],7))
print(time.time()-start_time)

1592.4611942768097


In [26]:
#16多线程
array_list2 = []
manager = mp.Manager()

p = Pool(16)

start_time = time.time()
def multi_get_sub_index(i):
    return get_sub_index(i)

tmp_result = []

for i in indices:
    result = p.apply_async(func=multi_convert_df_to_dm,args=(i,idds_uniq_sorted[:19],7,))
    tmp_result.append(result)
p.close()
p.join()

for item in tmp_result:
    array_list2.append(item.get())

print(time.time()-start_time)

180.80443000793457


In [28]:
sum = 0
new_indices = []
for i,item in enumerate(array_list2):
    if np.isnan(item).any():
        print(i)
#         sum+=1
    else:
        new_indices.append(i)

28681
28682
28683
28684
28690
28691
28692
28720
28768
28769
28770
28771
28772
28842
28851
28852
28853
29068
29069
29143
29144
29233
29234
29261
29304
29305
29330
29404
29405
29406
29407
29408
29409
29410
29449
29450
29451
29452
29453
29454
29455
29557
29558
29559
29560
29561
29604
29605
29606
29640
29641
29665
29673
29674
29794
29795
29882
29883
29884
29885
29886
29887
29888
29890


In [29]:
time_stamp_array

array([1162393768, 1162393798, 1162393828, ..., 1178896078, 1178896108,
       1178896138])

In [32]:
with open('4_tensor.pkl','wb') as f:
    pickle.dump(array_list2,f)

In [33]:
with open('4_time_stamp_array.pkl','wb') as f:
    pickle.dump(time_stamp_array,f)

In [34]:
with open('4_indices_na.pkl','wb') as f:
    pickle.dump(indices,f)

In [35]:
with open('4_indices_no_na.pkl','wb') as f:
    pickle.dump(new_indices,f)